In [5]:
from bert_serving.client import BertClient
#from bert_serving.server.helper import get_args_parser
#from bert_serving.server import BertServer
#args = get_args_parser().parse_args(['-model_dir', '/data/fjs/edp/chinese_L-12_H-768_A-12/',
#                                     '-port', '5555',
#                                     '-port_out', '5556',
#                                     '-max_seq_len', None,
#                                     '-num_worker','4'])
#server = BertServer(args)
#server.start()
#bert-serving-start -model_dir /data/comcode/bert/chinese_L-12_H-768_A-12/ -num_worker=2 -max_seq_len=None
bc = BertClient()
vec=bc.encode(['First do it', 'then do it right', 'then do it better'])
print (len(vec[0]))
print ((vec[1][0]))
#BertServer.shutdown(port=5555)

/usr/local/lib/python3.6/dist-packages/bert_serving/client/__init__.py:277: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


768
0.37245116


In [21]:
import pandas as pd
import numpy as np
import json
from bert_serving.client import BertClient
import torch
import torch.nn as nn
from torchvision import models,transforms
from torch.autograd import Variable
from PIL import Image
import time

#1.预训练resnet50模型构建
transform1 = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
resnet50_feature_extractor = models.resnet50(pretrained = True)
resnet50_feature_extractor.fc = nn.Linear(2048, 2048)
torch.nn.init.eye(resnet50_feature_extractor.fc.weight)
for param in resnet50_feature_extractor.parameters():
    param.requires_grad = False

#图向量化
img =Image.open("/data/tmpexec/1.png")
imgarr = np.array(img)
if len(imgarr.shape) == 2: #单通道转三通道
    c = [] 
    for i in range(3): c.append(imgarr) 
    imgarr = np.array(c)
    imgarr = imgarr.transpose([1,2,0])
    img = Image.fromarray(imgarr.astype('uint8')).convert('RGB')
elif imgarr.shape[2] == 4:#四通道转为三通道
    img = img.convert("RGB")
else:img =img #三通道
img = transform1(img)
x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)
y = resnet50_feature_extractor(x)
y = y.data.numpy()
print (y.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.eye is now deprecated in favor of nn.init.eye_.


(1, 2048)


In [2]:
import pandas as pd
import numpy as np
import time
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote,urlparse
import string
from PIL import Image
from bert_serving.client import BertClient
import torch
import torch.nn as nn
from torchvision import models,transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt

#习题加ID
#data = pd.read_csv("/data/fjsdata/medall.csv",sep='|',low_memory=False)
#data.drop_duplicates(inplace=True)#去重
#data.dropna(inplace=True)#去空行
#data['qid'] = data.index
#print ('数据集有%d行，%d列'%(data.shape[0],data.shape[1]))
#data.to_csv("/data/fjsdata/medallN.csv",index=False,sep='|')

starttime = time.time()
#1.加载数据集
data = pd.read_csv("/data/fjsdata/medallN.csv",sep='|',low_memory=False)#iterator=True
#data = data.get_chunk(200)
data.drop_duplicates(inplace=True)#去重
data.dropna(inplace=True)#去空行
data['TxtImg']=data['subType'].str.cat(data[['subCon','queType','queCon','queAnsA','queAnsB','queAnsC','queAnsD','queAnsE','quePar']],sep=',')
data=data[['qid','TxtImg']]
print ('原始数据集有%d行，%d列'%(data.shape[0],data.shape[1]))
#2预训练resnet50模型构建和Bert server连接
#2.1 resnet模型
transform1 = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
resnet50_feature_extractor = models.resnet50(pretrained = True)
resnet50_feature_extractor.fc = nn.Linear(2048, 2048)
torch.nn.init.eye(resnet50_feature_extractor.fc.weight)
for param in resnet50_feature_extractor.parameters():
    param.requires_grad = False
#2.2 bert模型
bc = BertClient()#连接BERT Server
#3.文本和图像信息解析并向量化函数
#3.1 文本
def getTxt(TxtImg):
    p=re.compile(r'[<](.*?)[>]', re.S) 
    matches = re.findall(p,TxtImg)
    for match in matches:
        TxtImg = TxtImg.replace('',"<"+match+">")
    return TxtImg
#3.2图像
def getImg(TxtImg):
    #提取链接
    p=re.compile(r'[<](.*?)[>]', re.S) 
    matches = re.findall(p,TxtImg)
    imgurls=[]
    for match in matches:
        if match.find('img')==0:#包含html代码，img在字符串首位
            soup = BeautifulSoup("<"+match+">")
            img = soup.find_all('img')
            src=img[0].get('src')
            imgurls.append(src)
        else:imgurls.append(match)#不包含html代码  
    #图片下载
    images=[]
    for imgurl in imgurls:
        try:
            parseret=urlparse(imgurl)#解析url格式
            if parseret.scheme == 'http':
                imgurl = quote(imgurl, safe=string.printable)#处理url中的中文字符
                img = Image.open(urlopen(imgurl))
                images.append(img)
        except Exception:pass
        continue
        
    if len(images)==0:return np.zeros((1,2048))
    else:
        #多图拼接成一张长图
        width,height=images[0].size
        for image in images:
            w,h = image.size
            if w>width:width=w
            if h>height: height=h
        longImg =  Image.new(images[0].mode,(width,height*len(images)))
        for i,im in enumerate(images):
            longImg.paste(im,box=(0,i*height))
        #plt.imshow(longImg)
        #plt.show()
        #长图向量化    
        imgarr = np.array(longImg)
        if len(imgarr.shape) == 2: #单通道转三通道
            c = [] 
            for i in range(3): c.append(imgarr) 
            imgarr = np.array(c)
            imgarr = imgarr.transpose([1,2,0])
            longImg = Image.fromarray(imgarr.astype('uint8')).convert('RGB')
        elif imgarr.shape[2] == 4:#四通道转为三通道
            longImg = longImg.convert("RGB")
        else:longImg =longImg #三通道
        longImg = transform1(longImg)
        x = Variable(torch.unsqueeze(longImg, dim=0).float(), requires_grad=False)
        y = resnet50_feature_extractor(x)
        return y.data.numpy() #2048长度
#4.向量化
#vectxt =data['TxtImg'].apply(lambda x: bc.encode([str(getTxt(x))])[0])#提取文本
vectxt =data['TxtImg'].apply(lambda x: bc.encode([str(x)])[0])#提取文本
data = pd.concat([data,pd.DataFrame(vectxt.tolist())],axis=1)#768维
vecimg =data['TxtImg'].apply(lambda x: getImg(str(x))[0])#提取图片链接
data = pd.concat([data,pd.DataFrame(vecimg.tolist())],axis=1)#2048维
data=data.drop(['TxtImg'], axis=1)
data.to_csv("/data/fjsdata/medallV.csv",index=False,sep='|')
print ('训练数据集有%d行，%d列'%(data.shape[0],data.shape[1]))
endtime = time.time()
print ("Complete time: %f s" % (endtime - starttime))

原始数据集有22366行，2列


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: nn.init.eye is now deprecated in favor of nn.init.eye_.
/usr/local/lib/python3.6/dist-packages/bert_serving/client/__init__.py:277: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


训练数据集有22366行，2817列
Complete time: 9930.258804 s


In [14]:
#相似度计算
import time
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from scipy import sparse
starttime = time.time()
#1.加载数据集
data = pd.read_csv("/data/fjsdata/mededp/medallV.csv",sep='|',low_memory=False)#,iterator=True)
#data = data.get_chunk(5)
print ('原始数据集有%d行，%d列'%(data.shape[0],data.shape[1]))
#2.余弦相似度计算
dist = 1-pairwise_distances(data, metric="cosine")
dist = pd.DataFrame(dist)
dist.to_csv("/data/fjsdata/mededp/medallSS.csv",index=False,header=,sep='|')
print ('相似数据集有%d行，%d列'%(dist.shape[0],dist.shape[1]))
#data_sparse = sparse.csr_matrix(data)
#sim_sparse = cosine_similarity(data_sparse,dense_output=False)
#print(pd.DataFrame(sim_sparse.todense()))
endtime = time.time()
print ("Complete time: %f s" % (endtime - starttime))

原始数据集有22366行，2817列
相似数据集有22366行，22366列
Complete time: 1184.368180 s


In [7]:
#linux只用\n换行，win下用\r\n表示换行。linux正常的一行在win下分多行，在字符串中替换\r
import pandas as pd
import numpy as np
data = pd.read_csv("/data/fjsdata/mededp/medallN.csv",sep='|',low_memory=False)#iterator=True
data['queCon']=data['queCon'].apply(lambda x:x.replace('\r','').replace('\n',''))#替换win下的分行符
data['queAnsA']=data['queAnsA'].apply(lambda x:x.replace('\r','').replace('\n',''))#替换win下的分行符
data['queAnsB']=data['queAnsB'].apply(lambda x:x.replace('\r','').replace('\n',''))#替换win下的分行符
data['queAnsC']=data['queAnsC'].apply(lambda x:x.replace('\r','').replace('\n',''))#替换win下的分行符
data['queAnsD']=data['queAnsD'].apply(lambda x:x.replace('\r','').replace('\n',''))#替换win下的分行符
data['queAnsE']=data['queAnsE'].apply(lambda x:x.replace('\r','').replace('\n',''))#替换win下的分行符
data['quePar']=data['quePar'].apply(lambda x:x.replace('\r','').replace('\n',''))#替换win下的分行符
#data = data.get_chunk(10)
print(data.shape)
print(data.head())
data.to_csv("/data/fjsdata/mededp/medallNN.csv",index=False,sep='|')

(22366, 11)
  subType          subCon queType  \
0   心血管内科  常见症状与体征(A1型题1)     单选题   
1   心血管内科  常见症状与体征(A1型题1)     单选题   
2   心血管内科  常见症状与体征(A1型题1)     单选题   
3   心血管内科  常见症状与体征(A1型题1)     单选题   
4   心血管内科  常见症状与体征(A1型题1)     单选题   

                                              queCon            queAnsA  \
0  女性，56岁，剧烈持续胸痛6小时入院。心电图示V1～6ST段明显上抬。双肺闻及湿性啰音，首选...  A.哌替啶，氨茶碱，低分子右旋糖酐   
1  男性，58岁，休息时突然出现右侧肢体无力，继而出现呼之不应。查体：深昏迷，压眶无反应，左侧瞳...              A.降颅压   
2        男性，32岁，有十二指肠球部溃疡病史8年，一天前出现柏油便。检查：贫血。其贫血的类型为           A.大细胞性贫血   
3  消化性溃疡病史患者，近日感上腹部疼痛加重，规律不明显，1小时前突然出现上腹部剧烈疼痛并放射至...          A.消化性溃疡穿孔   
4            女性，38岁，剧烈咳嗽后突然出现呼吸困难，胸透显示左侧气胸，查体可出现的体征有            A.呼吸音消失   

                queAnsB                   queAnsC  \
0  B.吗啡，呋塞米(原称速尿)，多巴酚丁胺  C.毒毛旋花子苷K，呋塞米(原称速尿)，倍他洛克   
1                 B.降血压                      C.吸氧   
2           B.小细胞低色素性贫血               C.小细胞高色素性贫血   
3                B.胆系疾病                     C.肾结石   
4                B.语颤增强                   C.叩诊过清音

In [2]:
import pandas as pd
data = pd.read_csv("/data/fjsdata/mededp/medallV.csv",sep='|',low_memory=False)
data = data[['qid']]
data.to_csv("/data/fjsdata/mededp/medqid.csv",sep='|')
print (data)

         qid
0          0
1          1
2          2
3          3
4          4
5          5
6          6
7          7
8          8
9          9
10        10
11        11
12        12
13        13
14        14
15        15
16        16
17        17
18        18
19        19
20        20
21        21
22        22
23        23
24        24
25        25
26        26
27        27
28        28
29        29
...      ...
22336  28332
22337  28333
22338  28334
22339  28335
22340  28336
22341  28337
22342  28338
22343  28339
22344  28340
22345  28341
22346  28342
22347  28343
22348  28344
22349  28345
22350  28346
22351  28347
22352  28348
22353  28349
22354  28350
22355  28351
22356  28352
22357  28353
22358  28354
22359  28355
22360  28356
22361  28357
22362  28358
22363  28377
22364  28378
22365  28379

[22366 rows x 1 columns]


In [3]:
import pandas as pd
import numpy as np
import re
def getImg(TxtImg):
    #提取链接
    p=re.compile(r'[<](.*?)[>]', re.S) 
    matches = re.findall(p,TxtImg)
    if len(matches)>0:print (matches) 
data = pd.read_csv("/data/fjsdata/mededp/medallNN.csv",sep='|',low_memory=False)#iterator=True
data['queCon']=data['queCon'].apply(lambda x:getImg(x))#找有图片的题干

['http://h5.img.zhukao.com.cn/Z/%E5%85%A8%E5%9B%BD/Z_%E5%85%A8%E5%9B%BD_%E5%BF%83%E8%A1%80%E7%AE%A1%E5%86%85%E7%A7%91/images/YZ71_~9.gif', 'http://h5.img.zhukao.com.cn/Z/%E5%85%A8%E5%9B%BD/Z_%E5%85%A8%E5%9B%BD_%E5%BF%83%E8%A1%80%E7%AE%A1%E5%86%85%E7%A7%91/images/YZ71_~9.gif']
['http://h5.img.zhukao.com.cn/Z/%E5%85%A8%E5%9B%BD/Z_%E5%85%A8%E5%9B%BD_%E5%BF%83%E8%A1%80%E7%AE%A1%E5%86%85%E7%A7%91/images/YZ71_~9.gif']
['http://h5.img.zhukao.com.cn/Z/%E5%85%A8%E5%9B%BD/Z_%E5%85%A8%E5%9B%BD_%E5%BF%83%E8%A1%80%E7%AE%A1%E5%86%85%E7%A7%91/images/~131.gif']
['http://h5.img.zhukao.com.cn/Z/%E5%85%A8%E5%9B%BD/Z_%E5%85%A8%E5%9B%BD_%E5%BF%83%E8%A1%80%E7%AE%A1%E5%86%85%E7%A7%91/images/~9.gif']
['http://h5.img.zhukao.com.cn/Z/%E5%85%A8%E5%9B%BD/Z_%E5%85%A8%E5%9B%BD_%E5%BF%83%E8%A1%80%E7%AE%A1%E5%86%85%E7%A7%91/images/~9.gif']
['http://h5.img.zhukao.com.cn/Z/%E5%85%A8%E5%9B%BD/Z_%E5%85%A8%E5%9B%BD_%E5%BF%83%E8%A1%80%E7%AE%A1%E5%86%85%E7%A7%91/images/~9.gif']
['http://h5.img.zhukao.com.cn/Z/%E5%85%A8%E5%9